# MLOps Example

Lorem

## 1. Setup

Lorem

In [ ]:
import json

import requests
import pandas as pd
from sklearn import *

from api.ifood import mlops

In [ ]:
project_id = mlops.create_project(name='iris')

print(project_id)

In [ ]:
projects = mlops.get_projects()

for project in projects:
    for k, v in project.items():
        print(f"{k}: {v}")
    print("")

## 2. Experiment

Lorem

In [ ]:
dataset = pd.read_csv('./feature-store/iris.csv')

In [ ]:
dataset["target"] = dataset["class"].astype("category").cat.codes
dataset.drop("class", axis=1, inplace=True)
dataset.head()

In [ ]:
train, test = model_selection.train_test_split(dataset, stratify=dataset["target"], random_state=42)

In [ ]:
experiment_id = mlops.run_experiment(
    project_id=project_id,
    engine='sklearn',
    model=linear_model.LogisticRegression(max_iter=1000),
    metrics=dict(accuracy=metrics.accuracy_score),
    target_col='target',
    train_data=train,
    test_data=test
)

print(experiment_id)

In [ ]:
experiment = mlops.get_experiment(experiment_id=experiment_id)

for k, v in experiment.items():
    print(f"{k}: {v}")
print(experiment['status'])

In [ ]:
experiment = mlops.get_experiment(experiment_id=experiment_id)

print(f"status: {experiment['status']}")

In [ ]:
experiments = mlops.get_experiments()

for experiment in experiments:
    for k, v in experiment.items():
        print(f"{k}: {v}")
    print("")

In [ ]:
experiment = mlops.get_experiment(experiment_id=experiment_id)

print(f"metrics: {experiment['metrics']}")

## 3. Deploy

Lorem

In [ ]:
mlops.deploy_experiment(experiment_id=experiment_id)

In [ ]:
experiment = mlops.get_experiment(experiment_id=experiment_id)

print(f"status: {experiment['status']}")

In [ ]:
deployments = mlops.get_deployments()

for deployment in deployments:
    for k, v in deployment.items():
        print(f"{k}: {v}")
    print("")

## 4. Predict

Lorem

In [ ]:
data = dict(model="iris", features={"sepal-length":5.7, "sepal-width": 3.8, "petal-length": 1.7, "petal-width": 0.3})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "0" for Iris-setosa

In [ ]:
data = dict(model="iris", features={"sepal-length":5.8, "sepal-width": 2.7, "petal-length": 4.1, "petal-width": 1.0})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "1" for Iris-versicolor

In [ ]:
data = dict(model="iris", features={"sepal-length":7.7, "sepal-width": 3.0, "petal-length": 6.1, "petal-width": 2.3})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "2" for Iris-virginica